# 컴파일러에서 변수, 조건문 다루기
TODO

In [1]:
data Expr = Var Name
          | Val Value
          | Add Expr Expr
          -- | Sub Expr Expr
          -- | Mul Expr Expr
          -- | Div Expr Expr
          | If Expr Expr Expr
          deriving Show

type Name = String
type Value = Int

data Inst = ADD | PUSH Value      -- 스택 명령
          | GOTO Code | JMPZ Code -- 실행코드 명령
          | READ Addr             -- 메모리 명령
          deriving Show
type Code = [Inst]

type Addr = Int

type Env = [ (Name, Value) ]
type SymTbl = [ (Name, Addr) ]
type Memory = [ (Addr, Value) ]

type Stack = [Value]
type Kont = (Stack,Memory,Code) -> (Stack,Memory,Code)


In [11]:
haltK :: Kont
haltK (s, mem, _) = (s, mem, [])

-- 스택 명령
pushK :: Int -> Kont
pushK n (s, mem, code) = (n:s, mem, code)

addK :: Kont
addK (n2:n1:s, mem, code) = ((n1+n2):s, mem, code)

-- 실행코드 명령
jmpzK :: Code -> Kont
jmpzK code (0:s, mem, _) = (s, mem, code)
jmpzK _    (_:s, mem, c) = (s, mem, c)

gotoK :: Code -> Kont
gotoK code (s, mem, _) = (s, mem, code)

-- 메모리 명령
readK a (s, mem, code) = case lookup a mem of
                           Nothing -> error (show a ++ " uninitialized memory address")
                           Just v  -> (v:s, mem, code)


지금 내가 주목해서 컴파일하는 부분의 목적 코드를 생성해서 
그 다음에 할 일의 코드 앞에다 이어붙이는
코드변환함수를 만드는 것이 컴파일러다

In [12]:
type Control = Code -> Code -- 코드변환함수
-- compile :: SymTbl -> Expr -> Control

compile :: SymTbl -> Expr -> Code
compile tbl (Var x) = case lookup x tbl of
                        Nothing -> error (x ++ " not found")
                        Just a  -> [READ a]
compile tbl (Val n) = [PUSH n]
compile tbl (Add e1 e2) = compile tbl e1 ++ compile tbl e2 ++ [ADD]
compile tbl (If e e1 e0) =
    compile tbl e ++ [JMPZ c0] ++ c1 ++ [GOTO []] ++ c0
    where
      c1 = compile tbl e1
      c0 = compile tbl e0

step :: Inst -> Kont
step (PUSH n) = pushK n
step ADD      = addK
step (GOTO c) = gotoK c
step (JMPZ c) = jmpzK c
step (READ a) = readK a

run :: Kont
run (s, mem, []) = (s, mem, [])
run (s, mem, c:cs) = run (step c (s, mem, cs))

In [13]:
import Data.List (union)

vars (Var x) = [x]
vars (Val _) = []
vars (Add e1 e2) = vars e1 `union` vars e2
vars (If e e1 e0) = vars e `union` vars e1 `union` vars e0

In [29]:
code0 = compile tbl0 e0
code0

[READ 102,READ 103,ADD,PUSH 100,ADD]

In [28]:
e0 = Add (Add (Var "x") (Var "y")) (Val 100)
e0

Add (Add (Var "x") (Var "y")) (Val 100)

In [31]:
vm0 = ([], [(102,7), (103,3)],code0)

run vm0

([110],[(102,7),(103,3)],[])

In [48]:
{-  b = ?,  x = 2, y = 4

    if b then (x + 3) else y  -}

e1 = If (Var "b") (Add (Var "x") (Val 3)) (Var "y")
e2 = e1 `Add` Val 1000

In [49]:
tbl0 = [("b",101),("x",102),("y",103)]
tbl0

mem0 = [(101,2), (102,12), (103,123)]
mem0

[("b",101),("x",102),("y",103)]

[(101,2),(102,12),(103,123)]

In [50]:
code1 = compile tbl0 e1
code1

code2 = compile tbl0 e2
code2

[READ 101,JMPZ [READ 103],READ 102,PUSH 3,ADD,GOTO [],READ 103]

[READ 101,JMPZ [READ 103],READ 102,PUSH 3,ADD,GOTO [],READ 103,PUSH 1000,ADD]

In [51]:
vm1 = ([], mem0, code1)
run vm1
vm2 = ([], mem0, code2)
run vm2

([15],[(101,2),(102,12),(103,123)],[])

([15],[(101,2),(102,12),(103,123)],[])

In [ ]:
code1 = compile tbl0 e1
code1

code2 = compile tbl0 e2
code2


import GHC.HeapView

putStr =<< ppHeapGraph <$> buildHeapGraph 15 code2 (asBox code2)

In [ ]:
vm0@(s0,   _,c0:cs0) = ([], mem0, code2)
vm0
vm1@(s1,mem1,c1:cs1) = step c0 (s0,mem0,cs0)
vm1
vm2@(s2,mem2,c2:cs2) = step c1 (s1,mem1,cs1)
vm2
-- vm3@(s3,mem3,c3:cs3) = step c2 (s2,mem2,cs2)
-- vm3

run vm0